In [1]:
%pwd

'd:\\Projects\\Alzeimer\\Alzheimer-s-Disease-Detection\\research'

In [2]:
import os
os.chdir("../")
%pwd

'd:\\Projects\\Alzeimer\\Alzheimer-s-Disease-Detection'

In [60]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

physical_devices = tf.config.list_physical_devices('GPU')

tf.test.is_gpu_available(
    cuda_only=True, min_cuda_compute_capability=None
)
tf.config.get_visible_devices(
    device_type=None
)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [61]:
BATCH_SIZE = 32
IMAGE_SIZE = 128
CHANNELS=3


In [62]:
train_path =r'artifacts\data_ingestion\train'
test_path = r'artifacts\data_ingestion\test'

In [63]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    seed=12,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    validation_split=0.05,  # Specify the validation split ratio
    subset="training",
    labels="inferred",
    label_mode="categorical"  # Specify that this is the training set
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    seed=12,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    validation_split=0.05,  # Specify the validation split ratio
    subset="validation",
    labels="inferred",
    label_mode="categorical"  # Specify that this is the validation set
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    seed=12,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    labels="inferred",
label_mode="categorical",

)

Name_of_the_classes = train_dataset.class_names

train_ds = train_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = validation_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

Found 5121 files belonging to 4 classes.
Using 4865 files for training.
Found 5121 files belonging to 4 classes.
Using 256 files for validation.
Found 1279 files belonging to 4 classes.


In [64]:
Name_of_the_classes = train_dataset.class_names

In [65]:
Name_of_the_classes

['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

In [66]:
'''plt.figure(figsize=(10,10))
for image_batch, laabels_batch in validation_dataset.take(1):
    print(image_batch.shape)
    print(laabels_batch.numpy())
    for i in range(12):
        ax = plt.subplot(4,3,i+1)
        plt.axis('off')
        plt.title(Name_of_the_classes[laabels_batch[i]])
        plt.imshow(image_batch[i].numpy().astype('uint8'))'''

"plt.figure(figsize=(10,10))\nfor image_batch, laabels_batch in validation_dataset.take(1):\n    print(image_batch.shape)\n    print(laabels_batch.numpy())\n    for i in range(12):\n        ax = plt.subplot(4,3,i+1)\n        plt.axis('off')\n        plt.title(Name_of_the_classes[laabels_batch[i]])\n        plt.imshow(image_batch[i].numpy().astype('uint8'))"

In [67]:
train_ds.element_spec


(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))

In [68]:
train_dataset.element_spec

(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))

In [69]:
resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.Rescaling(1./255)
])
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import math
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

import keras
from tensorflow import keras
from keras import Sequential
from keras import layers
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [71]:
import ssl

# Disable SSL verification
ssl._create_default_https_context = ssl._create_unverified_context


In [72]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.optimizers import Adam
from keras.applications import EfficientNetB0,VGG19
dropout_rate = 0.2
activation = "relu"
input_shape = (BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS)
base_model = VGG19(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
base_model.trainable = False

model = tf.keras.models.Sequential([
    resize_and_rescale,
    data_augmentation,
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Flatten(),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(256, activation=activation),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(128, activation=activation),
            tf.keras.layers.Dropout(dropout_rate),
          
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(
    
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.AUC(name='auc')])
             

In [74]:
def checkpoint_callback():
    
    checkpoint_filepath = 'checkpoint'
    
    model_checkpoint_callback= ModelCheckpoint(filepath=checkpoint_filepath,
                           save_weights_only=False,
                           frequency='epoch',
                           monitor='val_accuracy',
                           save_best_only=True,
                           verbose=1)
    
    return model_checkpoint_callback

    

def early_stopping(patience):
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=1)
    return es_callback


EPOCHS = 5
checkpoint_callback = checkpoint_callback()
early_stopping = early_stopping(patience=5)
callbacks = [checkpoint_callback, early_stopping]

In [75]:

history = model.fit(train_ds, epochs = EPOCHS, validation_data = val_ds, callbacks = callbacks)

Epoch 1/5
153/153 [==============================] - 1052s 7s/step - loss: 1.2091 - auc: 0.7508 - val_loss: 1.0965 - val_auc: 0.8004
Epoch 2/5
153/153 [==============================] - 963s 6s/step - loss: 1.0753 - auc: 0.7815 - val_loss: 1.0200 - val_auc: 0.8061
Epoch 3/5
 33/153 [=====>........................] - ETA: 12:20 - loss: 1.0631 - auc: 0.7876

KeyboardInterrupt: 

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import numpy as np
for images_batch, labels_batch in test_ds.take(1):
    
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",Name_of_the_classes[first_label])
    
    batch_prediction = model.predict(images_batch)
    print("predicted label:",Name_of_the_classes[np.argmax(batch_prediction[0])])

In [ ]:

def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = Name_of_the_classes[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence


In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = Name_of_the_classes[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

In [ ]:
import os

model.save("models1.h5")